# Load the datasets

In [21]:
import pandas as pd

# Load  sales dataset
sales_data = pd.read_excel("Ace Superstore Retail Dataset.xlsx", sheet_name="in")

# Load store location
store_locations = pd.read_excel("Store Locations.xlsx", sheet_name="Store Locations")

print("Sales Data:")
display(sales_data.head())

print("\nStore Locations:")
display(store_locations.head())


Sales Data:


,Order ID,Order Date,Order Mode,Customer ID,City,Postal Code,Country,Region,Product ID,Product Name,Category,Sub-Category,Sales,Cost Price,Quantity,Discount
0,BTC-245712,6/22/2024,Online,LO028977,Norton,S8,England,NaN,01JZ3N512GHNAJSF3HCCQ0PQYY,Flavored Popcorn Mix,Food - Snacks,Gourmet Snacks,2.99,0.897,4,0.24
1,NZR-891212,3/15/2024,In-Store,OH046670,West End,DN36,England,East of England,01JZ3N56DDVK7Y600QGH2M6T1X,Olive Oil,Food - Condiments,Cooking Oils,7.99,2.397,16,0.32
2,TDN-811093,2/29/2024,Online,FH015492,Newtown,RG20,England,East Midlands,01JZ3N53A64TW72TVK28SMMXKX,Children's Backpack,Accessories,Kids' Bags,29.99,8.997,2,0.09
3,MIX-746378,10/25/2024,In-Store,ZZ015342,East End,BH21,England,North West,01JZ3N572S4RVA29Y33YRWH174,Honey Butter Popcorn,Food - Snacks,Gourmet Snacks,2.99,0.897,4,0.01
4,UGI-201465,8/18/2024,Online,TV075977,Seaton,LE15,England,East Midlands,01JZ3N52RT7CJNB27BFS6H8BF7,Sliced Cucumbers,Food - Produce,Fresh Cut Vegetables,1.29,0.387,11,0.34



Store Locations:


,City,Postal Code,Country,Region
0,Birmingham,B12,England,West Midlands
1,Birmingham,B40,England,West Midlands
2,Thorpe,BD23,England,Yorkshire & the Humber
3,Bradford,BD7,England,Yorkshire & the Humber
4,East End,BH21,England,North West


# Explore sales data structure

In [22]:
print("Sales Dataset Info:")
sales_data.info()

print("\nStore Locations Info:")
store_locations.info()

# Missing values in each column
print("\nMissing values in sales_data:")
print(sales_data.isnull().sum())

print("\nMissing values in store_locations:")
print(store_locations.isnull().sum())

Sales Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      11000 non-null  object 
 1   Order Date    11000 non-null  object 
 2   Order Mode    11000 non-null  object 
 3   Customer ID   11000 non-null  object 
 4   City          11000 non-null  object 
 5   Postal Code   11000 non-null  object 
 6   Country       10780 non-null  object 
 7   Region        8189 non-null   object 
 8   Product ID    11000 non-null  object 
 9   Product Name  11000 non-null  object 
 10  Category      10802 non-null  object 
 11  Sub-Category  11000 non-null  object 
 12  Sales         11000 non-null  float64
 13  Cost Price    11000 non-null  float64
 14  Quantity      11000 non-null  int64  
 15  Discount      10027 non-null  float64
dtypes: float64(3), int64(1), object(12)
memory usage: 1.3+ MB

Store Locations Info:
<class 'pan

# Fill missing Country and Region using Postal Code from store_locations


In [23]:
sales_data['Postal Code'] = sales_data['Postal Code'].astype(str)
store_locations['Postal Code'] = store_locations['Postal Code'].astype(str)

# Merge to get Region and Country from store_locations
sales_data = sales_data.merge(
    store_locations[['Postal Code', 'Region', 'Country']],
    on='Postal Code',
    how='left',
    suffixes=('', '_store')
)

# Fill missing Country and Region in sales_data using the 'store' values
sales_data['Country'] = sales_data['Country'].fillna(sales_data['Country_store'])
sales_data['Region'] = sales_data['Region'].fillna(sales_data['Region_store'])

# Drop columns
sales_data.drop(columns=['Country_store', 'Region_store'], inplace=True)

# Rechecking missing values
print("Missing values AFTER filling Region and Country:")
print(sales_data[['Country', 'Region']].isnull().sum())


Missing values AFTER filling Region and Country:
Country    0
Region     0
dtype: int64


# Fill missing Category based on Sub-Category

In [24]:
# sub-category : category maps from rows that have both values
subcategory_to_category = sales_data[sales_data['Category'].notnull()].groupby('Sub-Category')['Category'].agg(lambda x: x.mode()[0])

# Fills missing categories using this map
sales_data['Category'] = sales_data.apply(
    lambda row: subcategory_to_category[row['Sub-Category']] if pd.isna(row['Category']) and row['Sub-Category'] in subcategory_to_category else row['Category'],
    axis=1
)

# missing Category values
print("Missing Category values AFTER mapping:", sales_data['Category'].isnull().sum())


Missing Category values AFTER mapping: 31


In [25]:
# Filter rows where Category is still missing
missing_category_rows = sales_data[sales_data['Category'].isnull()]

# Check which Sub-Categories that has missing values
remaining_subcategories = missing_category_rows['Sub-Category'].unique()

print("Sub-Categories that still have missing Category values:")
print(remaining_subcategories)


Sub-Categories that still have missing Category values:
['Seasoning Mixes' 'Dog Supplies' 'Audio Recording Devices' 'Seeds & Nuts'
 'Healthy Meals']


In [26]:
# Manually filling known logical exceptions (which auto-mapping missed)
fix_map = {
    'Seasoning Mixes': 'Food - Spices',
    'Dog Supplies': 'Pets',
    'Audio Recording Devices': 'Audio',
    'Seeds & Nuts': 'Food - Snacks',
    'Healthy Meals': 'Food - Meals'
}

sales_data['Category'] = sales_data.apply(
    lambda row: fix_map[row['Sub-Category']] 
    if pd.isna(row['Category']) and row['Sub-Category'] in fix_map 
    else row['Category'],
    axis=1
)

print("Missing Category values AFTER manual fix:", sales_data['Category'].isnull().sum())

Missing Category values AFTER manual fix: 0


In [27]:
# Fills missing discount values with 0.0
sales_data['Discount'] = sales_data['Discount'].fillna(0.0)

print("Remaining missing values in Discount:", sales_data['Discount'].isnull().sum())


Remaining missing values in Discount: 0


In [30]:
sales_data.to_csv("Ace_Superstore_Data1.csv", index=False)